In [14]:
from dotenv import load_dotenv
import os
import shioaji as sj
import pandas as pd
from clickhouse_connect import create_client
load_dotenv()

api = sj.Shioaji(simulation=True)
api.login(os.getenv("SINOPAC_KEY"),os.getenv('SINOPAC_SECRET'))
db = create_client(username='admin', password=os.getenv("BROKER_PASS"))
# api.fetch_contracts(True)

Response Code: 0 | Event Code: 0 | Info: host '210.59.255.161:80', hostname '210.59.255.161:80' IP 210.59.255.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up
Response Code: 200 | Event Code: 16 | Info: APISUB/V1/SYS/CONTRACT | Event: Subscribe or Unsubscribe ok


In [ ]:
list = [api.Contracts.Futures.TXF.TXFR1,api.Contracts.Futures.MXF.MXFR1]
api.Contracts.Futures
ticks=api.ticks(contract=list[1],date="2025-09-12")


In [72]:
df=pd.DataFrame({**ticks})
df.ts = pd.to_datetime(df.ts)

df = df[df['ts'] >= pd.to_datetime("2025-09-12 03:27:00")]
df = df[['ts','close', 'volume','tick_type']]
df.head(20)
# df.ts>pd.to_datetime("2025-09-12 03:27:00")

,ts,close,volume,tick_type
44333,2025-09-12 03:27:01.241,25407.0,2,1
44334,2025-09-12 03:27:12.929,25407.0,1,1
44335,2025-09-12 03:27:21.230,25407.0,1,1
44336,2025-09-12 03:27:30.303,25406.0,1,2
44337,2025-09-12 03:27:48.496,25407.0,1,1
44338,2025-09-12 03:27:59.176,25407.0,1,1
44339,2025-09-12 03:28:12.809,25406.0,1,2
44340,2025-09-12 03:28:12.915,25406.0,1,2
44341,2025-09-12 03:28:14.127,25407.0,1,1
44342,2025-09-12 03:28:14.394,25407.0,2,1


In [49]:
from collections import defaultdict

def process_orderflow(df, symbol, market_type,minute_offset):
    # df['ts'] = pd.Timestamp(df['ts'])
    df.set_index('ts', inplace=True)

    # Add side column: 1 = buy, -1 = sell, 0 = neutral
    df['side'] = df['tick_type'].map({1: 1, 2: -1, 0: 0})

    # Round timestamps to minute
    df['minute'] = df.index.floor('min')

    bars = []

    for minute, group in df.groupby('minute'):
        o = group['close'].iloc[0]
        h = group['close'].max()
        l = group['close'].min()
        c = group['close'].iloc[-1]
        v = group['volume'].sum()

        # Delta calculations
        # delta_close = (group['volume'] * group['side']).sum()
        # delta_high = delta_close
        # delta_low = delta_close
        trades_delta = group['side'].sum()
        delta_high = 0
        delta_low = 0
        delta_close = 0
        # Price map aggregation
        price_map_dict = defaultdict(lambda: [0, 0, 0])  # neutral, buy, sell
        for _, row in group.iterrows():
            price = round(row['close'], 4)
            if row['tick_type'] == 0:
                price_map_dict[price][0] += row['volume']
            elif row['tick_type'] == 1:
                price_map_dict[price][1] += row['volume']
            elif row['tick_type'] == 2:
                price_map_dict[price][2] += row['volume']
            delta_close += row['volume']*row['side']
            delta_high =  max(delta_high, delta_close)
            delta_low = min(delta_low, delta_close)

        price_map = [
            (price, neutral, buy, sell)
            for price, (neutral, buy, sell) in price_map_dict.items()
        ]
        # print(minute)
        bar = (market_type,minute.tz_localize(minute_offset),symbol,round(o, 4),round(h, 4),round(l, 4),round(c, 4),int(v),
            int(delta_high),
            int(delta_low),
            int(delta_close),
            int(trades_delta),
            price_map
        )

        bars.append(bar)

    return bars

In [68]:
# df['ts'].iloc[0].tz_localize(60*60*8)
rows=process_orderflow(df.reset_index(),symbol='MTX00',market_type=2,minute_offset=60*60*8)
# db.query_np("select * from ohlcvDM order by time desc limit 1")
# db.in

In [75]:
rows
# df
db.insert('bar_pipe',rows)